In [202]:
import re
import pendulum
import calendar

li_p_1_y = [r"(.*)来年(.*月)"]
li_p_2_y = [r"(.*)再来年(.*月)"]
hosei_year = r"(.[0-9])年"

li_p_14 = [r"再来週.*[月火水木金土日]曜"]
# li_p_next = [r"次の.*[月火水木金土日]曜",r"今度.*[月火水木金土日]曜"]
li_p_7 = [r"来週.*[月火水木金土日]曜"]
li_p_0 = [r"今週.*[月火水木金土日]曜"]
li_today = [r"今日.*時"]
li_tommorow = [r"明日.*時"]
li_da_tommorow = [r"明後日.*時"]

li_p_12_ji = [r"午後(.*)時",r"夕方(.*)時",r"夜(.*)時",r"昼(.*)時"]
li_tokushu_ji = {"正午":"12時"}


y_p =  r'([12]\d{3})[/\-年.]'
# y_wa_p = r'(明治|大正|昭和|平成|令和)\d{1,2}年'
m_p = r'(0?[1-9]|1[0-2])[/\-月.]'
d_p =  r'([12][0-9]|3[01]|0?[1-9])'
# h_p =  r'((0?|1)[0-9]|2[0-3])[:時]'
h_p =  r'(0?|[0-9]|1[0-9]|2[0-9])[:時]'
min_p =  r'([0-5][0-9]|0?[0-9])'
jyoshi_li = [r".*から",r".*に",r".*　",r".* ",r".*分",r".*は"]

def nan_shu(pendulum_datetime):
    return calendar.weekday(pendulum_datetime.year, pendulum_datetime.month, pendulum_datetime.day)

def re_search(pattern,string):
    prog = re.compile(pattern)
    result = prog.search(string)
    if result:
        out = result
    else:
        out = False
    return out

def trans_yobi_to_date(a,n,next_week_opp=True,konsh_opp=False):
    if "日曜" in a:
        tg = pendulum.now().next(pendulum.SUNDAY)
    elif "月曜" in a:
        tg = pendulum.now().next(pendulum.MONDAY)
    elif "火曜" in a:
        tg = pendulum.now().next(pendulum.TUESDAY)
    elif "水曜" in a:
        tg = pendulum.now().next(pendulum.WEDNESDAY)
    elif "木曜" in a:
        tg = pendulum.now().next(pendulum.THURSDAY)
    elif "金曜" in a:
        tg = pendulum.now().next(pendulum.FRIDAY)
    elif "土曜" in a:
        tg = pendulum.now().next(pendulum.SATURDAY)

    if next_week_opp:#同じ週番号であれば、7日足す　→　必ず来週を出す　日曜日起点であることを注意
        tg_shu = nan_shu(tg)
        now_shu = nan_shu(pendulum.now())
        if tg_shu == now_shu:
            n=n+7
            
    if konsh_opp:#今週　→　同じ週
        tg_shu = nan_shu(tg)
        now_shu = nan_shu(pendulum.now())
        if tg_shu != now_shu:
            print("今週はマイナスの日だよ")
            n=n-7
        if tg.weekday() == pendulum.now().weekday():
            print("今日のことかと、、、")
            n=n-7
                    

    return tg.add(days=n).strftime('%m月%d')

def date_trans(string):
    flg = False
    for i in li_p_14:#再来週の月曜日、、とかを処理
#         print(i)
        result = re_search(i,string)
        if result:
            tg = trans_yobi_to_date(result.group(),7)
            string = string.replace(result.group(),tg+" ")
            flg = True
            break

    if not flg:
        for i in li_p_7:#来週の月曜日、、とかを処理
#             print(i)
            result = re_search(i,string)
            if result:
                tg = trans_yobi_to_date(result.group(),0)
                string = string.replace(result.group(),tg)
                flg = True
                break

    if not flg:
        for i in li_p_0:#今週の月曜日、、とかを処理
#             print(i)
            result = re_search(i,string)
            if result:
                tg = trans_yobi_to_date(result.group(),0,next_week_opp=False,konsh_opp=True)
                string = string.replace(result.group(),tg)
                flg = True
                break
                
    if not flg:
        for i in li_today:#今日の10時、、、とかを処理
#             print(i)
            result = re_search(i,string)
            if result:
                tg = pendulum.now().add(days=0).strftime('%m月%d')
                r_temp = re.search(i[:-3],result.group())
                string = string.replace(r_temp.group(),tg)
                flg = True
                break

    if not flg:
        for i in li_tommorow:#明日の10時、、、とかを処理
#             print(i)
            result = re_search(i,string)
            if result:
                tg = pendulum.now().add(days=1).strftime('%m月%d')
                r_temp = re.search(i[:-3],result.group())
                string = string.replace(r_temp.group(),tg)
                flg = True
                break

    if not flg:
        for i in li_da_tommorow:#明後日の10時、、、とかを処理
#             print(i)
            result = re_search(i,string)
            if result:
                tg = pendulum.now().add(days=2).strftime('%m月%d')
                r_temp = re.search(i[:-3],result.group())
                string = string.replace(r_temp.group(),tg)
                flg = True
                break
    return string

def hun_hosei(string):
    result_1 = re_search(h_p,string)#時間でマッチ
    result_2 = re_search(h_p+min_p,string)#時間と分でマッチ
    result_3 = re_search(h_p+"半",string)#時間半があるかを確認

    if not result_1:
        pass
#         print("時間が指定されてないです")

    elif result_1 and not result_2:
#         print("時間があって、分がないので、半があれば30分、その他は0分にします")
        if result_3:
            string = string.replace(result_3.group(),result_1.group()+"30分")
        else:
            string = string.replace(result_1.group(),result_1.group()+"0分")

    return string

def year_shori(string):

    for i in  li_p_2_y:#再来年は今の年+2年
        if re_search(i,string):
            y = str(pendulum.now().add(years=2).year)+"年"
            a = r'\1 '+y+ r'\2'
            print(a)
            string = re.sub(i,a,string)[1:]

    for i in  li_p_1_y:#来年は今の年+1年（再来年を先に処理する）
        if re_search(i,string):
            y = str(pendulum.now().add(years=1).year)+"年"
            a = r'\1 '+y+ r'\2'
            string = re.sub(i,a,string)[1:]

    #年が2桁の場合は2000を足すにする
    result = re_search(y_p,string)
    if not result:
        nen = re_search(hosei_year,string)
        if nen:
            nen = re.sub(r"\D", "", nen.group())
            if len(nen)==2:
                nen_temp = int(nen)+2000
                string = string.replace(nen,str(nen_temp))
    #年がない場合は今年にする
    result = re_search(y_p,string)
    if not result:
        if re_search(m_p,string):
            string = str(pendulum.now().year)+"年"+string
            datetime_st = re_search(y_p+m_p,string).group().replace(" ","")
            y,datetime_st = toridasi(datetime_st,y_p)
            M,datetime_st = toridasi(datetime_st,m_p)
            delta = (pendulum.datetime(int(y), int(M), 1, 0, 0, 0) - pendulum.now()).days
            if delta < 0:
                print("来年の話ですね！")
                string = string.replace(str(pendulum.now().year)+"年",str(pendulum.now().year+1)+"年")
            else:
                pass
        else:
            string = str(pendulum.now().year)+"年"+string
    return string

def time_shori(string):
    for word, read in li_tokushu_ji.items():#正午のような特殊文字を処理
        string = string.replace(word, read)

    string = hun_hosei(string)#半を３０分とか、丁度の場合0分で補正

    for i in li_p_12_ji:#午後とか、昼のなどを処理
        result = re_search(i,string)
        if result:
            jikan = re.findall(i,result.group())[0]
            jikan = result.group().replace(jikan,str(int( re.sub(r"\D", "", jikan))+12))
            string = string.replace(result.group(),jikan)
            
    if not re_search(h_p+min_p,string):#時間が入ってない場合は、7：00にセット
        result = re_search(y_p+m_p+d_p,string)
        if result:
            string = string.replace(result.group(),result.group()+"日 7時0分")
    
    return string

def toridasi(datetime_st,seiki):
    result = re_search(seiki,datetime_st)
    return re.findall(seiki,datetime_st)[0],datetime_st[result.end():]

def main(string):
    string = string.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))#半角に変換

    string = year_shori(string)#年の処理
    string = date_trans(string)#月と日の処理
    string = time_shori(string)#時間の処理
    print(string)
    datetime = re_search(y_p+m_p+d_p+".*"+h_p+min_p,string)
    if datetime:
        datetime_st = datetime.group().replace(" ","")
        datetime_st = datetime_st.replace("　","")
        y,datetime_st = toridasi(datetime_st,y_p)
        M,datetime_st = toridasi(datetime_st,m_p)
        d,datetime_st = toridasi(datetime_st,d_p)
        h,datetime_st = toridasi(datetime_st,h_p)
        mi,datetime_st = toridasi(datetime_st,min_p)

        naiyo = string[datetime.end():]
        for i in jyoshi_li:
            result = re_search(i,naiyo)
            if result:
                naiyo = naiyo[result.end():]
            else:
                naiyo = naiyo
        out = y,M,d,h,mi,naiyo
    else:
        print("失敗")
        out = False,string,datetime
    return out

In [207]:
out = main("1/23は桜花さんの誕生日")
out

来年の話ですね！
2022年1/23日 7時0分は桜花さんの誕生日


('2022', '1', '23', '7', '0', '桜花さんの誕生日')